In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl 

In [2]:
sc.settings.verbosity=3
sc.logging.print_versions()

scanpy==1.5.1 anndata==0.7.1 umap==0.3.10 numpy==1.17.3 scipy==1.4.1 pandas==0.25.3 scikit-learn==0.22.1 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


In [11]:
## pip install fa2

In [13]:
## pip install pynndescent 

# Read data

In [3]:
fn="/nfs/rprdata/julong/SCAIP/analyses/SCAIP_pseudotime_2022.6.14/0_data.outs/1_Tcell.PHA.h5ad"
adata=sc.read_h5ad(fn)

In [4]:
print(adata.n_vars)
print(adata.n_obs)
adata

6571
29157


AnnData object with n_obs × n_vars = 29157 × 6571 
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'total_counts', 'NEW_BARCODE', 'SNG.BEST.GUESS', 'treat2', 'MCls'
    var: 'features'

In [5]:
adata.X.shape

(29157, 6571)

In [7]:
iid=pd.read_csv("./0_data.outs/0.1_iid.csv", header=0, index_col=False)

In [8]:
cells=adata.obs['NEW_BARCODE']

In [9]:
cells

SCAIP5V3-PHA-DEX_AAACCCAAGGACGCAT    SCAIP5V3-PHA-DEX_AAACCCAAGGACGCAT
SCAIP5V3-PHA-DEX_AAACCCACAAATACGA    SCAIP5V3-PHA-DEX_AAACCCACAAATACGA
SCAIP5V3-PHA-DEX_AAACCCACATGAGAAT    SCAIP5V3-PHA-DEX_AAACCCACATGAGAAT
SCAIP5V3-PHA-DEX_AAACCCATCACCATGA    SCAIP5V3-PHA-DEX_AAACCCATCACCATGA
SCAIP5V3-PHA-DEX_AAACCCATCGTTCAGA    SCAIP5V3-PHA-DEX_AAACCCATCGTTCAGA
                                                   ...                
SCAIP6B-PHA-EtOH_TTTGTTGTCAAAGGTA    SCAIP6B-PHA-EtOH_TTTGTTGTCAAAGGTA
SCAIP6B-PHA-EtOH_TTTGTTGTCAAATGCC    SCAIP6B-PHA-EtOH_TTTGTTGTCAAATGCC
SCAIP6B-PHA-EtOH_TTTGTTGTCCCTAGGG    SCAIP6B-PHA-EtOH_TTTGTTGTCCCTAGGG
SCAIP6B-PHA-EtOH_TTTGTTGTCCGGTAAT    SCAIP6B-PHA-EtOH_TTTGTTGTCCGGTAAT
SCAIP6B-PHA-EtOH_TTTGTTGTCCGTGTAA    SCAIP6B-PHA-EtOH_TTTGTTGTCCGTGTAA
Name: NEW_BARCODE, Length: 29157, dtype: object

# Simulation

In [14]:
##
pseu_DF=[]
i=0
for i in range(50):
    ##
    print(i)
    id=iid.iloc[:,i]-1
    cellSel=cells[id]
    adata2=adata[cellSel,:]
    ## pre-process
    sc.pp.recipe_zheng17(adata2)
    sc.tl.pca(adata2, svd_solver="arpack")
    sc.pp.neighbors(adata2, n_neighbors=4, n_pcs=30)
    ## diffusion map
    sc.tl.diffmap(adata2, n_comps=10)
    sc.pp.neighbors(adata2, n_neighbors=10, method='gauss', use_rep='X_diffmap')
    ## pseudotime
    adata2.uns['iroot']=np.flatnonzero(adata2.obs['treat2']=='PHA')[0]
    sc.tl.dpt(adata2)
    ##
    pseu=adata2.obs['dpt_pseudotime']
    pseu_DF.append(pseu)

0
running recipe zheng17
filtered out 77 genes that are detected in less than 1 counts


Trying to set attribute `.var` of view, copying.


normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
    finished (0:00:01)
computing PCA
    with n_comps=50
    finished (0:00:03)
computing neighbors
    using 'X_pca' with n_pcs = 30


/wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/numba/compiler.py:588: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../../wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  self.func_ir.loc))
/wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.py

    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:11)
computing Diffusion Maps using n_comps=10(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         0.9977936  0.992199   0.99117637 0.9905016  0.98677343
     0.98634195 0.98588014 0.98518467 0.9848408 ]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing neighbors


...

normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
    finished (0:00:00)
computing PCA
    with n_comps=50
    finished (0:00:03)
computing neighbors
    using 'X_pca' with n_pcs = 30


/wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/numba/compiler.py:588: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../../wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:02)
computing Diffusion Maps using n_comps=10(=n_dcs)
computing transitions
    finished (0:00:00)
    eigenvalues of transition matrix
    [1.         1.         0.9964165  0.9925605  0.9917945  0.9914428
     0.9905955  0.9903583  0.9887529  0.98861897]
    finished: added
    'X_diffmap', diffmap coordinates (adata.obsm)
    'diffmap_evals', eigenvalues of transition matrix (adata.uns) (0:00:00)
computing neighbors


/wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/numba/compiler.py:588: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../../wsu/home/groups/piquelab/apps/el7/anaconda3python/envs/scanpy/lib/python3.6/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:09)
computing Diffusion Pseudotime using n_dcs=10
    finished: added
    'dpt_pseudotime', the pseudotime (adata.obs) (0:00:00)


In [15]:
pseu2=[list(pseu) for pseu in pseu_DF]

In [16]:
## generate data frame
names=["id"+str(i) for i in list(range(1,51))]
pseu_DF2=pd.DataFrame(pseu2).transpose()
pseu_DF2.columns=names

In [17]:
### output
opfn="./2_scanpy.outs/1_pseudotime.csv"
pseu_DF2.to_csv(opfn,index=False)


In [ ]:
pseu_DF2.shape

In [ ]:
## diffusion map
##sc.tl.diffmap(adata2, n_comps=10)
##sc.pp.neighbors(adata2, n_neighbors=10, method='gauss', use_rep='X_diffmap')

In [ ]:
## pseudotime
##adata2.uns['iroot']=np.flatnonzero(adata2.obs['treat2']=='PHA')[0]
##sc.tl.dpt(adata2)